In [1]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: <object object at 0x7f4a4ea4e510>
2.0.0+cu117


'%.4f'

In [2]:
def get_mnist2():
    x, y = lib.datasets.get_mnist_train()
    train_y = y % 2
    train_x = torch.reshape(x, (len(x), -1))
    x, y = lib.datasets.get_mnist_test()
    test_y = y % 2
    test_x = torch.reshape(x, (len(x), -1))
    return train_x, train_y

dataset_func = lib.datasets.get_linear3

if True or "train_x" not in globals():
    train_x, train_y = dataset_func()



In [3]:
def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.optimizer.LiveNetOptimizer(net, lr=0.01)
        # optimizer = torch.optim.Adam(net.parameters())
    else:
        print("Torch")
        # optimizer = lib.optimizer.optimizer_with_lr_property(torch.optim.SGD, net.parameters(), lr=0.01)
        optimizer = lib.optimizer.optimizer_with_lr_property(torch.optim.Adam, net.parameters(), betas=(0.0, 0.95))
    return optimizer


In [4]:
lib.utils.set_seed()
# network = lib.nets.LINEAR3(l1=0.1)
network = lib.nets.create_livenet_linear3(l1=0.1)
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=len(train_x))
criterion = lib.nets.criterion_n
optimizer = create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=30, adaptive_lr=False)


LiveNet


In [7]:

trainer.step(200)


Iˈ0.000 0.581 = 0.215+0.366 lib/trainer.py:89
Iˈ0.077 0.581 = 0.215+0.366 lib/trainer.py:89
Iˈ0.159 0.581 = 0.215+0.366 lib/trainer.py:89
Iˈ0.251 0.581 = 0.215+0.366 lib/trainer.py:89
Iˈ0.313 0.581 = 0.215+0.366 lib/trainer.py:89
Iˈ0.360 0.581 = 0.215+0.366 lib/trainer.py:89


In [8]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.0000